In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
def preprocess(df, split_column, drop_first=True):
    '''
    Does the usual preprocessing steps on a pandas dataframe:
    1. Does one-hot encoding
    2. Standardizes the data
    3. Drop N/A
    4. Splits the data into x & y along split_column
    5. Returns (x, y)
    '''
    stds = StandardScaler().set_output(transform='pandas')
    df = stds.fit_transform(pd.get_dummies(df, drop_first=drop_first)).dropna()
    x = df.drop(split_column, axis=1)
    return (x, df[split_column][x.index])

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect('resources/database.sqlite') 

In [4]:
df2 = pd.read_sql_query("SELECT * FROM player_attributes INNER JOIN Player ON Player.id = player_attributes.id", conn)
df2.shape

(11060, 49)

In [5]:
df1 = pd.read_csv('resources/final_data.csv')
df1.rename(columns={'name': 'player_name'}, inplace=True)

In [6]:
merged_df = pd.merge(df1, df2, on='player_name')
merged_df = merged_df.dropna()  # Drop rows with missing values
merged_df = merged_df.drop_duplicates(subset='player_name')  

merged_df.shape

(1220, 70)

In [7]:
merged_df['goal_per_appearance'] = merged_df['goals'] / merged_df['appearance']
merged_df['assist_per_appearance'] = merged_df['assists'] / merged_df['appearance']

In [8]:
merged_df.head()

,player,team,player_name,position,height_x,age,appearance,goals,assists,yellow cards,...,gk_positioning,gk_reflexes,id,player_api_id,player_fifa_api_id,birthday,height_y,weight,goal_per_appearance,assist_per_appearance
0,/jack-butland/profil/spieler/128899,Manchester United,Jack Butland,Goalkeeper,196.0,30.0,15,0.000000,0.000000,0.069018,...,69.0,68.0,4502,288880,203042,1993-03-10 00:00:00,195.58,209,0.000000,0.000000
1,/tom-heaton/profil/spieler/34130,Manchester United,Tom Heaton,Goalkeeper,188.0,37.0,4,0.000000,0.000000,0.000000,...,75.0,78.0,10377,24155,163264,1986-04-15 00:00:00,187.96,190,0.000000,0.000000
2,/phil-jones/profil/spieler/117996,Manchester United,Phil Jones,Defender Centre-Back,185.0,31.0,8,0.000000,0.000000,0.216346,...,68.0,72.0,8589,186137,194957,1992-02-21 00:00:00,185.42,157,0.000000,0.000000
3,/luke-shaw/profil/spieler/183288,Manchester United,Luke Shaw,Defender Left-Back,178.0,27.0,74,0.015374,0.169115,0.353604,...,13.0,8.0,6432,362694,205988,1995-07-12 00:00:00,185.42,165,0.000208,0.002285
4,/casemiro/profil/spieler/16306,Manchester United,Casemiro,midfield-DefensiveMidfield,185.0,31.0,101,0.090817,0.124874,0.329213,...,70.0,75.0,1596,208494,200145,1992-02-23 00:00:00,182.88,176,0.000899,0.001236


In [9]:
merged_df.to_csv("resources/test.csv", index=False)

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor


(X, y) = preprocess(merged_df, 'goals')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

#Baseline
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_train, y_train)
y_dummy_pred = dummy.predict(X_test)
baseline_mse = mean_squared_error(y_test, y_dummy_pred)
print(f'Baseline MSE: {baseline_mse}')



Mean Squared Error: 0.6695191272128337
Baseline MSE: 0.934561805631509


In [11]:
print(y.describe())

count    1211.000000
mean        0.005662
std         1.001956
min        -0.761815
25%        -0.761815
50%        -0.392546
75%         0.384680
max         9.695349
Name: goals, dtype: float64


In [12]:
rmse = np.sqrt(0.0008381504680398298)
print("RMSE:", rmse)

RMSE: 0.028950828451701167


In [13]:
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

B = np.arange(10,200,10)
grid = {'n_estimators':B}

rf = GridSearchCV(RandomForestRegressor(),param_grid=grid,return_train_score=True,n_jobs=-1)
rf.fit(X_train,y_train)

cv_results = cross_validate(rf,X_train,y_train,return_train_score=True)
R2_trainCV = cv_results['train_score'].mean()
R2_valid   = cv_results['test_score'].mean()
predictions = rf.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print('train R2 (CV) =',R2_trainCV,'  valid R2 =',R2_valid)
print()
R2_train = rf.score(X_train,y_train)
R2_test  = rf.score(X_test,y_test)
print('     train R2 =',R2_train,'    test R2 =',R2_test)
print('mse = ' + str(mse))
print('mae = ' + str(mae))

train R2 (CV) = 0.9850467657647355   valid R2 = 0.9006410241892511

     train R2 = 0.98388293913725     test R2 = 0.9227570587164161
mse = 0.07216240002290004
mae = 0.10240794986736662
